In [5]:
import pandas as pd
from difflib import SequenceMatcher
import os
import os
import subprocess
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector

In [6]:
# read files
df = pd.read_csv('totalset.csv', compression = 'gzip')
pd.set_option('display.max_columns', None)
df = df.head()

In [7]:
df

,identifier,gt text,ocr text,CER,WER,WER (order independent),dictionary lookup gt,dictionary lookup ocr,jaccard_coefficient,levenshtein_dist_normalized,source,word count gt,word count ocr
0,ddd:010728187:mpeg21:a0005,"NEDERLANDEN. Valencijn, den 23 July. Sedert de...","NEDERLANDEN. Vakncijn, den j -> July. Scdeit d...",34.66,49.34,46.10,87.99,51.68,0.254021,0.22,Meertens,1968,1791
1,ddd:010728188:mpeg21:a0001,"VRANCKRYCK. PArijs, den 12 Augusti. Den Hertog...","VRANKRYK. PÜAriif , den 12 Den Hertog van Sund...",11.86,37.43,36.84,93.10,67.54,0.392045,0.11,Meertens,169,162
2,ddd:010728188:mpeg21:a0002,"DUYTSLANDT, &amp;c. Straetsburg, den 12 Aug. G...","DUYTSLANDT, & C. Strertil-H,?, den ft Gifteren...",31.87,70.23,66.05,87.05,55.03,0.163569,0.31,Meertens,215,187
3,ddd:010728188:mpeg21:a0003,NEDERLANDEN. Uyt het Leger onder sijne Hooghey...,NEDERLANDEN. iVl.t het. Leger tndtr fi'yne Hoo...,17.50,51.32,44.40,86.12,47.12,0.208452,0.17,Meertens,3254,3217
4,ddd:010731495:mpeg21:a0001,"Ordinarise Middel-weeckse Courante, No. 30. 16...","Uyt Madrid , den 30 Jury. Stère .ïliajeflept t...",49.03,84.17,83.33,92.63,47.73,0.091954,0.48,Meertens,118,98


In [8]:
def fuzzy_matching(row): # create files with matched sentences
    save_path = r"C:\Users\Gebruiker\Desktop\Thesis\Documenten\Documenten"
    
    identifier = row['identifier'].split(':')
    ID = identifier[1]
    page = identifier[3]
    
    # split GT and OCR-output of each row into sentences (based on full stops).
    gt_sentences = row['gt text'].split(".")
    ocr_sentences = row['ocr text'].split(".")

    match_list = []

    for sentence in gt_sentences:
        for match in ocr_sentences:
            if SequenceMatcher(None, sentence, match).ratio() > 0.75:
                match_list.extend([[sentence, match]])
                
    matched_gt_sentences = []
    matched_ocr_sentences = []

    for sentence in match_list:
        if len(sentence[0]) or len(sentence[1]) > 0:
            matched_gt_sentences.append(sentence[0])
            matched_ocr_sentences.append(sentence[1])
            
    # .join() with lists
    separator = '.'
    doc_GT = separator.join(matched_gt_sentences)
    doc_OCR = separator.join(matched_ocr_sentences)
    
    filename = f"{ID}_{page}_OCR.txt"
    file_ocr = open(os.path.join(save_path, filename),"w+", encoding="utf-8")
    file_ocr.write(doc_OCR)
    file_ocr.close()
    
    filename = f"{ID}_{page}_GT.txt"
    file_gt = open(os.path.join(save_path, filename),"w+", encoding="utf-8")
    file_gt.write(doc_GT)
    file_gt.close()
    
    return matched_gt_sentences, matched_ocr_sentences
    
    

In [9]:
def evaluation():
    path = r"C:\Users\Gebruiker\Desktop\Thesis\Documenten\Documenten"
    output_path = r"C:\Users\Gebruiker\Desktop\Thesis\Documenten\Evaluatie"
    location_jar_file = r"C:\Users\Gebruiker\Desktop\Thesis\Testbestanden\ocrevalUAtion-1.3.4-jar-with-dependencies.jar"

    # Iterate through all the Ground Truth files
    for i in range(0, len(os.listdir(path)), 2):
        filename = os.listdir(path)[i]
        # Split the filename, and extract the identifier and pagenr together as identifier 
        print(filename)
        identifier = filename.split('_')[0] + "_" + filename.split('_')[1]

        file_gt = path + "\\" + identifier + "_GT.txt"
        file_ocr = path + "\\" + identifier + "_OCR.txt"
        output = output_path + "\\" + identifier + ".html"

        # Run the ocrevalUAtion tool with given parameters:
        process = subprocess.call("java -cp " + location_jar_file  + " eu.digitisation.Main -gt " + file_gt + " -ocr "+ file_ocr +" -o " + output + "")

        # Let the program wait for a few seconds, so the system does not get overloaded
        sleep(5)
    
    
    #Create a dataframe to store the scores
    dfScore = pd.DataFrame(columns = ['identifier', 'CER', 'WER', 'WER (order independent)'])

    # Iterate through all the output files
    for filename in os.listdir(output_path):
        # Open the file
        soup = BeautifulSoup(open(output_path + "\\" + filename, encoding='utf-8'))
        table = soup.find("table", attrs={'border': '1'})
        # Split the filename, and extract the identifier and pagenr together as identifier 
        identifier = filename.split('_')[0] + "_" + filename.split('_')[1]
        # Find the first table (this is the table in which the scores are stored)
        # Find the tags in which 'CER', 'WER', and 'WER (order independent)' are stored and take the next tag to get the score 
        cer = table.find('td', text='CER')
        cerScore = cer.findNext('td')
        wer = table.find('td', text='WER')
        werScore = wer.findNext('td')
        werOI = table.find('td', text='WER (order independent)')
        werOIScore = werOI.findNext('td')
        # Add the score of the file, including its identifier, to the dataframe
        dfScore = dfScore.append({'identifier': filename, 'CER': cerScore.text, 'WER': werScore.text, 'WER (order independent)': werOIScore.text}, ignore_index = True)

    # Write the created dataframe to a .csv file
    #dfScore.to_csv('Score_ocr_evalution.csv', sep=';')
    print(dfScore)

In [89]:
# word alignment
def word_alignment(GT, OCR):
    shortest_list = min([GT, OCR], key=len)
    longest_list = max([GT, OCR], key=len)
    min_ratio = 0.65

    for i in range(len(longest_list)):
        match = False
        #i = -i-1
        # check if there is a direct match:
        try:
            if OCR[i] == ' ' and GT[i] == '':
                del OCR[i]
                del GT[i]
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= min_ratio:
                match = True
                continue
            elif match == False:
                # check if there is a match with a word before in OCR:
                try:
                    for j in range(1,3):
                        if i-j >= 0:
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i-j] == ''):
                                    for k in range(0,j):
                                        OCR.insert(i-j, '')
                                    match = True
                                    continue
                        if i+j <= len(OCR):          
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() >= min_ratio:
                                if not (GT[i] == '' and OCR[i+j] == ''):
                                    for k in range(0,j):
                                        GT.insert(i, '')
                                    match = True
                                    continue
                except:
                    pass
        except IndexError:
            min([GT, OCR], key=len).append('')

        if match == False:
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= 0.30:
                pass
            else:
                min([GT, OCR], key=len).insert(i, '')
                continue


    if len(GT) != len(OCR):
        # backwards loop

        shortest_list = min([GT, OCR], key=len)
        longest_list = max([GT, OCR], key=len)

        for i in range(len(longest_list)):
            i = -i-1
      
            try:

                if (OCR[i] == '') ^ (GT[i] == ''):

                    continue
                if OCR[i] == ' ' and GT[i] == '':
                    del OCR[i]
                    del GT[i]
                if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= min_ratio:
     
                    continue
                else:
                    # check if there is a match with a word before in OCR:
                    try:
                        for j in range(1,3):
                            if SequenceMatcher(None, GT[i], OCR[i-j]).ratio() >= min_ratio:
    
                                for k in range(0,j):
                                    if i+j != 0:
                                        GT.insert(i+j, '')
                                    else: 
                                        GT.append('')

                                break
                            if SequenceMatcher(None, GT[i], OCR[i+j]).ratio() >= min_ratio:

                                for k in range(0,j):
                                    GT.insert(i, '')

                                break
                    except:
                        pass

            except IndexError:
                min([GT, OCR], key=len).append('')

            # if there is no match:
            if SequenceMatcher(None, GT[i], OCR[i]).ratio() >= 0.50:
                pass
            else:
                if (OCR[i] == '') or (GT[i] == ''):
                    continue
                else:
                    min([GT, OCR], key=len).insert(i+1, '')

            

    return GT, OCR


In [90]:
# verify word alignment quality
def verify_alignment(GT, OCR):
    
    if len(GT) != len(OCR):
        alignment_validation = False
        return alignment_validation
    
    min_word_length = 8 # the higher this number, the more likely a alignment will be seen as valid, thus higher chance at false positive, lower chance at false negative. 
    match_ratio = 0.70
    anchors_list = []
    for i in range(len(GT)):
        if len(GT[i]) >= min_word_length:  
            anchors_list.append(GT[i])
        if len(OCR[i]) >= min_word_length:
            anchors_list.append(OCR[i])
    #print(anchors_list)


    alignment_validation = True

    for anchor in anchors_list:
        #print('ANCHOR:', anchor)
        matches_GT = set()
        matches_OCR = set()
        match = False
        for i in range(len(GT)):
            if (SequenceMatcher(None, GT[i], anchor).ratio() >= 0.65) and (len(GT[i]) >= min_word_length or len(GT[i]) == 0):
                matches_GT.add(i)
                #print('added as GT match:', GT[i])
            if SequenceMatcher(None, OCR[i], anchor).ratio() >= 0.65 and (len(OCR[i]) >= min_word_length or len(OCR[i]) == 0):
                matches_OCR.add(i)
                #print('added as OCR match:', OCR[i])
        #print('anchor:', anchor)
        #print(matches_GT)
        #print(matches_OCR)
        if matches_GT == matches_OCR:
            match = True
        if match == False:
            match = True
            points_set1 = set() # counts when the matched opposite word is '' for OCR
            points_set2 = set() # counts when the matched opposite word is '' for GT
            for index in matches_GT.difference(matches_OCR): # elements in matches_GT that are not in matches_OCR
                if OCR[index] == '':
                    points_set1.add(index)
                if OCR[index] != '' and len(matches_OCR.difference(matches_GT)) != 0:
                    match = False
            for index in matches_OCR.difference(matches_GT): # elements in matches_OCR that are not in matches_GT
                if GT[index] == '':
                    points_set2.add(index)
                if GT[index] != '' and len(matches_GT.difference(matches_OCR)) != 0:
                    match = False
            if len(points_set1.intersection(points_set2)) != 0: # if an anchor words was match in both OCR and GT with an '', it's a bad match.
                match = False
        if match == False:
            for index in (matches_GT.difference(matches_OCR)):
                index += 1
                if index in matches_OCR.difference(matches_GT):
                    if (GT[index-1] == '' or OCR[index-1] == '') and (SequenceMatcher(None, GT[index], OCR[index]).ratio() >= match_ratio):
                        match = True
            for index in (matches_OCR.difference(matches_GT)):
                index += 1
                if index in matches_GT.difference(matches_OCR):
                    if (GT[index] == '' or OCR[index] == '') and (SequenceMatcher(None, GT[index-1], OCR[index-1]).ratio() >= match_ratio):
                        match = True

        if match == False:
            alignment_validation = False

    return alignment_validation

In [91]:
def find_errors(GT, OCR): # uses the aligned data
    
    # find errors:
    error_list = []

    for i in range(len(GT)):
        error = []
        if len(GT) == len(OCR):
            if GT[i] != OCR[i]:
                if OCR[i] != '':
                    print(GT[i], OCR[i])
                    error.append(OCR[i])
                    error.append(GT[i])
                    error.append(i)
                    error_list.append(error)
            
    
    print('error_list:', error_list)
    
    #print(wrong_alignment)
    return error_list

In [92]:
# match sentences GT and OCR and create new documents
GT_matched_docs_list = []
OCR_matched_docs_list = []

for index, row in df.iterrows():
    matched_gt_sentences, matched_ocr_sentences = fuzzy_matching(row)
    GT_matched_docs_list.append(matched_gt_sentences)
    OCR_matched_docs_list.append(matched_ocr_sentences)


In [93]:
# calculate CER and WER new documents
evaluation()

010728187_a0005_GT.txt
010728188_a0001_GT.txt
010728188_a0002_GT.txt
010728188_a0003_GT.txt
010731495_a0001_GT.txt
             identifier    CER    WER WER (order independent)
0  010728187_a0005.html  13,99  38,44                   35,99
1  010728188_a0001.html  21,02  51,06                   44,68
2  010728188_a0002.html  15,22  61,54                   57,69
3  010728188_a0003.html  18,24  43,66                   40,38
4  010731495_a0001.html  31,58  25,00                   25,00


In [94]:
print(matched_docs_GT[1])

[['VRANCKRYCK'], ['', '', '', '', '', '', '', 'Den', '', 'Hertog', 'van', 'Sunderlant', 'continueert', 'hier', 'in', 'besoignes,', '', 'doende', 'groote', 'klachten', 'over', 'onse', 'Fregatten,', '', 'die', 'hebben', 'geweygert', 'te', 'strijcken', 'voor', 'sijn', 'Kon'], ['', 'Bergen', 'is', 'naeu', 'beset,', 'en', 'alle', 'Advenues', 'soodanig', 'gefortificeert,', 'dat', 'men', 'niet', 'vertrout,', 'dat', 'de', 'Geallieerden', 'yets', 'sullen', 'onderstaen', 'om', 'daer', 'in', 'te', 'brengen'], ['', '']]


In [95]:
matched_docs_GT = []
matched_docs_OCR = []

for i in range(len(GT_matched_docs_list)):
    matched_gt_sentences = GT_matched_docs_list[i]
    matched_ocr_sentences = OCR_matched_docs_list[i]

    aligned_GT = []
    aligned_OCR = []

    for j in range(len(matched_gt_sentences)):
        sentGT = matched_gt_sentences[j]
        sentOCR = matched_ocr_sentences[j]
        sentGT = sentGT.split(' ')
        sentOCR = sentOCR.split(' ')
        GT, OCR = word_alignment(sentGT, sentOCR)
        
        aligned_GT.append(GT)
        aligned_OCR.append(OCR)

    matched_docs_GT.append(aligned_GT)
    matched_docs_OCR.append(aligned_OCR)

In [96]:
# layers of data alignment list:
# matched_docs_GT/OCR = data alignment for all documents
# matched_docs_GT/OCR[i] = data alignment for one document
# matched_docs_GT/OCR[i][j] = data alignment for one sentence in one document
# matched_docs_GT/OCR[i][j][k] = data alignment for one word in once sentence in one document

In [97]:
# layers of data alignment list:
# matched_docs_GT/OCR = data alignment for all documents
# matched_docs_GT/OCR[i] = data alignment for one document
# matched_docs_GT/OCR[i][j] = data alignment for one sentence in one document
# matched_docs_GT/OCR[i][j][k] = data alignment for one word in once sentence in one document

good_alignments = []
bad_alignments = []

for i in range(len(matched_docs_GT)):
    # for each aligned document
    for j in range(len(matched_docs_GT[i])):
        #print('GT:', matched_docs_GT[i][j])
        #print('OCR:', matched_docs_OCR[i][j])
        # for each aligned sentence
        alignment_validation = verify_alignment(matched_docs_GT[i][j], matched_docs_OCR[i][j])
        alignment = [matched_docs_GT[i][j], matched_docs_OCR[i][j]]
        if alignment_validation == True:
            good_alignments.append(alignment)
        elif alignment_validation == False:
            bad_alignments.append(alignment)

print('good:', len(good_alignments))
print('bad:', len(bad_alignments))

good: 86
bad: 5


In [102]:
print(matched_docs_GT[0][1])
#matched_docs_GT[0][1][5]


['', 'Valencijn,', 'den', '', '', '23', 'July']


In [103]:
datasetlevel_error_list = []

# matched_docs_GT/OCR[i][j] = data alignment for one sentence in one document
for i in range(len(matched_docs_GT)):
    documentlevel_error_list = []
    for j in range(len(matched_docs_GT[i])):
        print(matched_docs_GT[i][j])
        print(matched_docs_OCR[i][j])
        error_list = find_errors(matched_docs_GT[i][j], matched_docs_OCR[i][j])
        documentlevel_error_list.append(error_list)
    datasetlevel_error_list.append(documentlevel_error_list)

print(datasetlevel_error_list)

['NEDERLANDEN']
['NEDERLANDEN']
error_list: []
['', 'Valencijn,', 'den', '', '', '23', 'July']
['', 'Vakncijn,', 'den', 'j', '->', '', 'July']
Valencijn, Vakncijn,
 j
 ->
error_list: [['Vakncijn,', 'Valencijn,', 1], ['j', '', 3], ['->', '', 4]]
['', 'Charleroy,', 'den', '24', 'July']
['', 'Chariêrty,', 'den', '24', 'Juiy']
Charleroy, Chariêrty,
July Juiy
error_list: [['Chariêrty,', 'Charleroy,', 1], ['Juiy', 'July', 4]]
['', 'Monsr']
['', 'Monlr']
Monsr Monlr
error_list: [['Monlr', 'Monsr', 1]]
['', 'Brussel,', 'den', '', '27', 'July']
['', 'Bruffél,', 'den', 'z', '7', 'July']
Brussel, Bruffél,
 z
27 7
error_list: [['Bruffél,', 'Brussel,', 1], ['z', '', 3], ['7', '27', 4]]
['', 'Den', '21', 'deser', 'zijn', 'de', 'Franse', '', 'voor', 'Arien', 'gekomen,', '', 'onder', 'den', '', 'Maerschalck', "d\\'Humieres,", 'ende', 'hadden', "d\\'onse", 'gedwongen', 'het', 'Fort', 'St']
['', 'Den', '11', 'defer', 'zijn', 'de', '', 'Fi', 'anfevoor', 'Arien', 'geicomen', ',', 'onder', 'den', 'i\\', 'a

In [104]:
# matched_docs_GT/OCR[i][j] = data alignment for one sentence in one document
wrong_length = 0
right_length = 0

for i in range(len(matched_docs_GT)):
    documentlevel_error_list = []
    for j in range(len(matched_docs_GT[i])):
        if len(matched_docs_GT[i][j]) != len(matched_docs_OCR[i][j]):
            #print('GT:', matched_docs_GT[i][j])
            #print('OCR:', matched_docs_OCR[i][j])
            wrong_length += 1
        else:
            right_length += 1

print(wrong_length)
print(right_length)
print(wrong_length / (wrong_length+right_length))


3
88
0.03296703296703297


In [105]:
# example of how to find errors
# the first document = GT_matched_docs_list[0], OCR_matched_docs_list[0]
# say: for each sentence, a random word is detected as an error:
import random
detected_error_list = []

for sentence in OCR_matched_docs_list[0]:
    detected_error = []
    sentence = sentence.split(' ')
    detected_error_index = random.randint(0, len(sentence)-1)
    #print(detected_error_index)
    #print(sentence[detected_error_index])
    detected_error.append(sentence[detected_error_index])
    detected_error.append(detected_error_index)
    detected_error_list.append(detected_error)

print('detected_error_list:', detected_error_list)

correct_errors = 0
incorrect_errors = 0

for i in range(len(detected_error_list)):
    #print(detected_error_list[i])
    #print(datasetlevel_error_list[0][i])
    correct_error = False
    for j in range(len(datasetlevel_error_list[0][i])):
        #print(datasetlevel_error_list[0][i][j])
        if detected_error_list[i][0] == datasetlevel_error_list[0][i][j][0]:
            print(f'same: {detected_error_list[i][0]}, {datasetlevel_error_list[0][i][j][0]}')
            print(f'index: {detected_error_list[i][1]}, {datasetlevel_error_list[0][i][j][2]}')
            correct_error = True
            print(f'The correct spelling of "{detected_error_list[i][0]}" is "{datasetlevel_error_list[0][i][j][1]}"')
            correct_errors += 1
    if correct_error == True:
        continue
    else:
        print(f'The word "{detected_error_list[i][0]}" is not actually an error.')
        incorrect_errors += 1
    
            # problem: the indexes are different because one index is from the fuzzy matched sentences, the other from
            # the aligned words.
print(f'Correct errors: {correct_errors}, incorrect errors: {incorrect_errors}, accuracy: {correct_errors/(correct_errors+incorrect_errors)}.')

detected_error_list: [['NEDERLANDEN', 0], ['July', 5], ['den', 2], ['', 0], ['7', 4], ['Fi', 6], ['vooriz', 7], ['te', 22], ['heeft', 10], ['Schans', 1], ['om', 12], ['dag', 17], ['twee', 6], ['Prins', 7], ['fijn', 22], ['gefontlieytj', 7], ['', 0], ['van', 21], ['heden', 24], [',', 2], ['25', 3], ['houden', 10], ['dat', 9], ['heeft', 16], ['laten', 20], [':de', 26], ['heden', 5], ['', 0], ['nu', 3], ['met', 6], ['', 0], ['cenige', 6], ['den', 2], ['D', 9], ['Ambafladeut', 10], ['foude', 2], ['', 1]]
The word "NEDERLANDEN" is not actually an error.
The word "July" is not actually an error.
The word "den" is not actually an error.
The word "" is not actually an error.
same: 7, 7
index: 4, 4
The correct spelling of "7" is "27"
same: Fi, Fi
index: 6, 7
The correct spelling of "Fi" is ""
same: vooriz, vooriz
index: 7, 7
The correct spelling of "vooriz" is "voorsz"
The word "te" is not actually an error.
The word "heeft" is not actually an error.
The word "Schans" is not actually an error.


In [106]:
/*

SyntaxError: can't use starred expression here (<ipython-input-106-9f8a92b391d5>, line 4)